In [1]:
import requests
import json
import pandas as pd
import random
import time
from datetime import datetime
from timeit import default_timer as timer
from requests_ip_rotator import ApiGateway, EXTRA_REGIONS, ALL_REGIONS

In [2]:
#Better readability for JSON output
def jprint(obj):
    text = json.dumps(obj, sort_keys=True, indent = 4)
    print(text)

In [3]:
def get_trades(tid_start, tid_end, currency):
    if tid_start >= tid_end: #make sure valid starting and ending values provided
        tid = tid_start #initialize 'tid' to starting value
        tmp_list = [] #temp list to hold dfs from loop
        trades = pd.DataFrame() #final df of trades
        i = 0 #while-loop counter for sleep function
        
        gateway = ApiGateway("https://localbitcoins.com", regions = ["us-west-2"]) #Use ApiGateway from requests_ip_rotator to randomize IP used for requests to avoid rate limiting
        gateway.start() #open gateway, always shutdown as well

        session = requests.Session() #begin session
        session.mount("https://localbitcoins.com", gateway)

        
        try:
            while (tid >= tid_end):
                
                parameters = {
                    'max_tid': tid #passes current value of 'tid' as a parameter into the API request
                }

                url = 'https://localbitcoins.com/bitcoincharts/' + str(currency) + '/trades.json'

                t = session.get(url, params = parameters) #pull last 500 trades, starting with max_tid
                t.raise_for_status() #Returns HTTP error object
                df = pd.DataFrame(t.json()) #get JSON output into pd DataFrame
                tmp_list.append(df.iloc[:-1]) #append to temp list, omits last row to avoid repeat obs
                
                tid_last = df['tid'].iloc[-1] #gets the last tid of the previous dataset, new starting point for next loop
                tid = tid_last
                time.sleep(1 + (random.randint(0, 1000) / 1000))#sleep function to avoid API errors, this one moderates time between API requests
                i += 1 #loop counter
                
                if i%500 == 0: #only for multiples of 500 (get API error around 800 without IP randomization)
                    # time.sleep(1200) #sleep for 20 minutes
                    print('congrats! you have made ' + str(i) + ' API requests, you insane person.')
                
                
                
        except Exception as e:
            print(t.status_code)
            print(e.response.text)
            print(tid)
            gateway.shutdown()
            trades = pd.concat(tmp_list, ignore_index=True) #concatinate all DFs in temp list into one dataframe
            trades['Currency'] = str(currency) #append the currency to a column
            return trades
        
        else:
            gateway.shutdown()
            trades = pd.concat(tmp_list, ignore_index=True) #concatinate all DFs in temp list into one dataframe
            trades['Currency'] = str(currency) #append the currency to a column
            return trades

    else:
        print('Please enter a value for tid_start that is greater than tid_end')

In [ ]:
currencies = ['USD']
start_tid = 56510558
end_tid = 6000000


for currency in currencies:
    print(currency)
    start = timer()
    
    df = get_trades(start_tid, end_tid, currency)
    path = '../temporary/trades_' + str(currency) + '.csv'
    end = timer()
    time = end - start
    print(time)
    
    df.to_csv(path, index = False)

USD
Starting API gateway in 1 regions.
Using 1 endpoints with name 'https://localbitcoins.com - IP Rotate API' (1 new).
congrats! you have made 500 API request, you insane person.
congrats! you have made 1000 API request, you insane person.
congrats! you have made 1500 API request, you insane person.
congrats! you have made 2000 API request, you insane person.


In [14]:
df

date       tid     price      amount Currency
0        1514143600  11382842  22999.93  0.00108696      USD
1        1514143355  11382840  22999.93  0.00108696      USD
2        1514142925  11382832  23664.03  0.00190162      USD
3        1514143060  11382827  25701.66  0.00233448      USD
4        1514143006  11382825  80817.22  0.00030934      USD
...             ...       ...       ...         ...      ...
1168154  1487818576   5999301   1520.83  0.24000000      USD
1168155  1487818250   5999295   1156.07  0.08650000      USD
1168156  1487818257   5999294   2783.96  0.17960000      USD
1168157  1487818543   5999292   1515.15  0.01320000      USD
1168158  1487818710   5999291   1358.33  0.36810000      USD

[1168159 rows x 5 columns]

In [5]:
datetime.fromtimestamp(1664127368)

datetime.datetime(2022, 9, 25, 11, 36, 8)

In [4]:
#See most recent trades
t = requests.get('https://localbitcoins.com/bitcoincharts/USD/trades.json')
jprint(t.json())

[
    {
        "amount": "0.03189204",
        "date": 1664330110,
        "price": "18029.58",
        "tid": 56510558
    },
    {
        "amount": "0.00553009",
        "date": 1664330110,
        "price": "21699.47",
        "tid": 56510556
    },
    {
        "amount": "0.00249436",
        "date": 1664330222,
        "price": "20045.22",
        "tid": 56510553
    },
    {
        "amount": "0.00307000",
        "date": 1664329958,
        "price": "18550.49",
        "tid": 56510549
    },
    {
        "amount": "0.00291974",
        "date": 1664330004,
        "price": "20549.77",
        "tid": 56510546
    },
    {
        "amount": "0.00476258",
        "date": 1664328608,
        "price": "20997.02",
        "tid": 56510531
    },
    {
        "amount": "0.00375645",
        "date": 1664329203,
        "price": "19965.66",
        "tid": 56510522
    },
    {
        "amount": "0.00091336",
        "date": 1664327994,
        "price": "21897.17",
        "tid": 565105